# Phases (adjust according to the proposed methodology)
### Dataset distribution


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
import tensorflow_datasets as tfds

# Descargar y preparar el dataset EuroSAT
dataset, info = tfds.load('eurosat', with_info=True, as_supervised=True)
train_dataset = dataset['train']
num_classes = info.features['label'].num_classes
class_names = info.features['label'].names
print(f'Número de clases: {num_classes}')
print(f'Clases: {class_names}')


### Baseline models (at least three architectures in PyTorch)


In [ ]:
# Transformaciones de imagen
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Preprocesar el dataset
def preprocess(dataset, transform):
    images = []
    labels = []
    for image, label in tfds.as_numpy(dataset):
        image = transform(image)
        images.append(image)
        labels.append(label)
    return torch.stack(images), torch.tensor(labels)

# Dividir el dataset en entrenamiento, validación y prueba
train_size = int(0.8 * len(train_dataset))
val_size = int(0.1 * len(train_dataset))
train_data, train_labels = preprocess(train_dataset.take(train_size), transform)
val_data, val_labels = preprocess(train_dataset.skip(train_size).take(val_size), transform)
test_data, test_labels = preprocess(train_dataset.skip(train_size + val_size), transform)

# Crear DataLoaders
batch_size = 32
train_loader = DataLoader(list(zip(train_data, train_labels)), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(list(zip(val_data, val_labels)), batch_size=batch_size)
test_loader = DataLoader(list(zip(test_data, test_labels)), batch_size=batch_size)


### Hyperparameter selection


In [ ]:
# Función para crear modelos base
def create_model(model_name):
    if model_name == 'resnet18':
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(4096, num_classes)
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model

# Crear modelo y definir parámetros
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 10
lr = 0.001
criterion = nn.CrossEntropyLoss()


### Model fine tuning


In [ ]:
# Entrenamiento del modelo
def train_model(model, train_loader, val_loader, epochs, lr):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_losses, val_losses = [], []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_losses.append(val_loss / len(val_loader))
        print(f'Epoch {epoch + 1}/{epochs}, Training Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}')
    return train_losses, val_losses


### Model evaluation


In [ ]:
# Evaluación del modelo
def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    print(classification_report(y_true, y_pred, target_names=class_names))
    conf_matrix = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('Etiqueta verdadera')
    plt.xlabel('Etiqueta predicha')
    plt.title('Matriz de confusión')
    plt.show()


In [ ]:
# Probar diferentes arquitecturas
for model_name in ['resnet18', 'vgg16', 'efficientnet_b0']:
    print(f'\nEntrenando con la arquitectura {model_name}')
    model = create_model(model_name)
    train_model(model, train_loader, val_loader, epochs, lr)
    print(f'\nEvaluando el modelo {model_name}')
    evaluate_model(model, test_loader)
